In [8]:
from datasets import get_dataset_config_names
from datasets import load_dataset
from huggingface_hub import whoami, login

import os
import csv
import time
from datasets import get_dataset_config_names, load_dataset
from tqdm.auto import tqdm

from datasets import Features,Value
from datasets import get_dataset_config_names

In [2]:
login()

In [3]:
print(whoami())

{'type': 'user', 'id': '6908a4546cb49b1f11079ac0', 'name': 'aadityakshatriya', 'fullname': 'Aaditya Kshatriya', 'email': 'ph22b004@smail.iitm.ac.in', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/9b0301f7a081ad87dbea60693cdba914.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'aadityastokenread', 'role': 'read', 'createdAt': '2025-11-03T13:05:38.838Z'}}}


In [ ]:
configs = get_dataset_config_names("ARTPARK-IISc/vaani", token=True)
print(configs)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

['AndhraPradesh_Anantpur', 'AndhraPradesh_Annamaya', 'AndhraPradesh_Chittoor', 'AndhraPradesh_Guntur', 'AndhraPradesh_Krishna', 'AndhraPradesh_SriSatyaSai', 'AndhraPradesh_Srikakulam', 'AndhraPradesh_Vishakapattanam', 'ArunachalPradesh_Longding', 'ArunachalPradesh_LowerDibangvalley', 'ArunachalPradesh_PapumPare', 'Assam_Hailakandi', 'Assam_KamrupMetropolitan', 'Assam_Sonitpur', 'Bihar_Araria', 'Bihar_Begusarai', 'Bihar_Bhagalpur', 'Bihar_Darbhanga', 'Bihar_EastChamparan', 'Bihar_Gaya', 'Bihar_Gopalganj', 'Bihar_Jahanabad', 'Bihar_Jamui', 'Bihar_Kaimur', 'Bihar_Katihar', 'Bihar_Kishanganj', 'Bihar_Lakhisarai', 'Bihar_Madhepura', 'Bihar_Muzaffarpur', 'Bihar_Patna', 'Bihar_Purnia', 'Bihar_Saharsa', 'Bihar_Samastipur', 'Bihar_Saran', 'Bihar_Sitamarhi', 'Bihar_Supaul', 'Bihar_Vaishali', 'Bihar_WestChamparan', 'Chandigarh_Chandigarh', 'Chhattisgarh_Balrampur', 'Chhattisgarh_Bastar', 'Chhattisgarh_Bilaspur', 'Chhattisgarh_Jashpur', 'Chhattisgarh_Kabirdham', 'Chhattisgarh_Korba', 'Chhattisgarh

In [12]:
import os
print(os.path.abspath(SAVE_DIR))

c:\IITM 4th year physics\ML\Sarvam AI pre winter project\Hindi_Dataset


In [1]:
import os
import json
import io
from datasets import load_dataset, get_dataset_config_names, Features, Value
import soundfile as sf
from tqdm.auto import tqdm

DATASET = "ARTPARK-IISc/vaani"
SAVE_DIR = "Hindi_With_Transcriptions"
os.makedirs(SAVE_DIR, exist_ok=True)

configs = get_dataset_config_names(DATASET)
configs = [c for c in configs if c not in ("default", "images")]

print(f"Found {len(configs)} regions to scan.")

total_seen = 0
saved = 0

for cfg in configs:
    print(f"\n🔍 Scanning region: {cfg}")

    # Stream to avoid downloading all audio
    try:
        ds = load_dataset(DATASET, cfg, split="train", streaming=True)

        # Ensure audio stays as raw bytes
        ds = ds.cast_column(
            "audio",
            Features({
                "path": Value("string"),
                "bytes": Value("binary")
            })
        )

    except Exception as e:
        print(f"❌ Could not stream {cfg}: {e}")
        continue

    # Iterate through samples
    for sample in tqdm(ds, desc=f"{cfg}"):
        total_seen += 1

        # Filter Hindi + has transcription
        if sample.get("language") != "Hindi":
            continue

        if str(sample.get("isTranscriptionAvailable", "No")).lower() != "yes":
            continue

        # Actually download audio bytes
        audio_bytes = sample["audio"]["bytes"]
        if audio_bytes is None:
            continue

        try:
            waveform, sr = sf.read(io.BytesIO(audio_bytes), dtype="float32")
        except:
            continue

        saved += 1
        file_id = f"{cfg}_{saved:06d}"

        # Save audio file
        wav_path = os.path.join(SAVE_DIR, f"{file_id}.wav")
        sf.write(wav_path, waveform, sr)

        # Save cleaned metadata (remove bytes!)
        meta = sample.copy()
        meta["audio"] = {"path": sample["audio"]["path"]}

        json_path = os.path.join(SAVE_DIR, f"{file_id}.json")
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(meta, f, indent=2)

    print(f"➡️ Region {cfg}: saved {saved} files so far.")

print("\n🎉 FINISHED")
print("Total samples checked:", total_seen)
print("Total Hindi with transcription saved:", saved)
print("Saved in:", SAVE_DIR)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Found 147 regions to scan.

🔍 Scanning region: AndhraPradesh_Anantpur


Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

AndhraPradesh_Anantpur: 0it [00:00, ?it/s]

...

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Bihar_Kishanganj: 0it [00:00, ?it/s]

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/ARTPARK-IISc/vaani/resolve/dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc/audio/Bihar/Kishanganj/train-00039-of-00042.parquet (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019C6BBF0190>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 52f868f3-4ba2-431c-832d-de0045d2c679)')' thrown while requesting GET https://huggingface.co/datasets/ARTPARK-IISc/vaani/resolve/dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc/audio/Bihar/Kishanganj/train-00039-of-00042.parquet
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/ARTPARK-IISc/vaani/resolve/dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc/audio/Bihar/Kishanganj/train-00039-of-00042.parquet (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019C6B857B10>, 'Connecti

ConnectionError: Server Disconnected